<a href="https://colab.research.google.com/github/Anurag-root/Anurag-root.github.io/blob/master/Trading_Strategy_Long.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Description: this program will give buy and sell indications by following the supertrend and 200 DMA crossover 

In [ ]:
#importing librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [ ]:
pip install pandas_ta

     |████████████████████████████████| 115 kB 35.2 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=e6b7683b6f0605218bc4e9191d8cda01f68c5cb07f9756ac2718947c79ac76e8
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [ ]:
import pandas_ta as ta
from google.colab import files
uploaded = files.upload()

Saving niftytoday.csv to niftytoday.csv


In [ ]:
# Store the data 
df = pd.read_csv('niftytoday.csv')
# Show the data
# df

In [ ]:
supertrend = ta.supertrend(high =df['high'], low =df['low'], close = df['close'], length= 7, multiplier = 3)['SUPERT_7_3.0']

In [ ]:
df['sup'] = supertrend

In [ ]:
# visually show figure
# plt.figure(figsize=(16,8))
# plt.title('close price nifty bank',fontsize=18)
# plt.plot(df['close'])
# plt.xlabel('Date',fontsize=18)
# plt.ylabel('close',fontsize=18)
# plt.show

In [ ]:
# create a function to calculate 200 day moving average 
def DMA (data, period=200 ,column = 'close'):
  return data[column].rolling(window= period).mean()

In [ ]:
# create a function to calculate 20 day moving average 
def DMA (data, period=20 ,column = 'close'):
  return data[column].rolling(window= period).mean()

In [ ]:
#  create a new column for dma 200
df['DMA200']= DMA(df,200) 

In [ ]:
# create a new column for dma 20
df['DMA20'] = DMA(df,20)

In [ ]:
# plt.figure(figsize=(12.2,4.5))
# plt.plot(df.close)
# plt.plot(df.sup, color='green')
# plt.plot(df.DMA200, color='red')
# plt.show()

In [ ]:
# plt.figure(figsize=(12.2,4.5))
# plt.plot(df.close)

# plt.plot(df.DMA20, color='red')
# plt.show()

In [ ]:
# Define entries whether its long or not for 200 dma
df['Long200'] = (df.sup > df.DMA200) & (df.close > df.sup)
df['Down200'] = (df.DMA200 > df.sup) & (df.close < df.DMA200)


In [ ]:
# Define entries whether its long or not for 20 dma
df['Long20'] = (df.close > df.DMA20) 
df['Short20'] = (df.DMA20 > df.close) 

In [ ]:
# get buy and sell signal
# df['Signal'] = np.where(df['sup'] > df['DMA200'] , 1 , 0)
# df['Position'] = df['Signal'].diff()
# df['Buy'] = np.where(df['Position'] == 1 ,df['close'] , np.NAN)
# df['Sell'] = np.where(df['Position'] == -1 ,df['close'] , np.NAN)

In [ ]:
# function when to buy and sell for sup and 200dma for banknifty 
def long_short(signal):
  Take =[]
  Leave =[]
  flag =-1

  for i in range (0,len(signal)):
    if (signal['sup'][i] > signal['DMA200'][i]) & (signal['sup'][i] < signal['close'][i]) :
      Leave.append(np.nan)
      if flag != 1:
        Take.append(signal['close'][i])
        flag = 1
      else:
        Take.append(np.nan)

    elif (signal['sup'][i] < signal['DMA200'][i]) & (signal['close'][i] < signal['DMA200'][i]):
      Take.append(np.nan)
      if flag != 0:
        Leave.append(signal['close'][i])
        flag = 0
      else:
        Leave.append(np.nan)

    else:
        Take.append(np.nan)
        Leave.append(np.nan)


  return (Take, Leave)

In [ ]:
# calling 200dma function
b = long_short(df)
df['Buy200'] = b[0]
df['Sell200'] = b[1]

In [ ]:
D_Red= df.Sell200.dropna()

In [ ]:
D_Green = df.Buy200.dropna()

In [ ]:
# only 20 dma function buy and sell
def buy_sell(signal):
  Buy =[]
  Sell =[]
  flag =-1

  # 50% candle 
  open = signal['open']
  close = signal['close']
  more_than_half = [0] * len(signal)


  # 

  for i in range (0,len(signal)  ): 
    more_than_half[i] = abs(open[i] - close[i] ) * 0.55 + close[i]
    # for Long signal 
    if ((signal['close'][i] > signal['DMA20'][i]) | (signal['DMA20'][i] < more_than_half[i])):
    # if(signal['DMA20'][i] < more_than_half[i]):
      Sell.append(np.nan)
      if flag != 1:
        Buy.append(signal['close'][i])
        flag = 1
      else:
        Buy.append(np.nan)

# for Short signal
    elif ((signal['close'][i] < signal['DMA20'][i]) | (signal['DMA20'][i] > more_than_half[i])):
    # elif (signal['DMA20'][i] > more_than_half[i]):
      Buy.append(np.nan)
      if flag != 0:
        Sell.append(signal['close'][i])
        flag = 0
      else:
        Sell.append(np.nan)

# for SL or Sell (in case of reversal)
    elif ((more_than_half[i] > signal['DMA20'][i]) & (signal['close'][i+1] < signal['close'][i])):
      Buy.append(np.nan)
      if flag != 0:
        Sell.append(signal['close'][i])
        flag = 0
      else:
        Sell.append(np.nan)

# for SL or Buy (in case of reversal)
    elif ((more_than_half[i] < signal['DMA20'][i]) & (signal['close'][i+1] > signal['close'][i])):
      Sell.append(np.nan)
      if flag != 1:
        Buy.append(signal['close'][i])
        flag = 1
      else:
        Buy.append(np.nan)
    
    else:
        Buy.append(np.nan)
        Sell.append(np.nan)
        

  return (Buy, Sell)


In [ ]:
# calling 20 dma function
a = buy_sell(df)
df['Buy_Signal_Price'] = a[0]
df['Sell_Signal_Price'] = a[1]




In [ ]:
Red = df.Sell_Signal_Price.dropna()

In [ ]:
Green = df.Buy_Signal_Price.dropna()

In [ ]:
import requests

In [ ]:
for sell in Red[-1:]:
  base_url = 'https://api.telegram.org/bot2049051709:AAGOApTYM_4ZMiSIpt_lvj4QDGrN8_X8MYc/sendMessage?chat_id=-542715306&text= "{}" :: @Banknifty (Short Activated by 20DMA)'.format(sell)
  requests.get(base_url)
  # print(base_url)

In [ ]:
for sell in D_Red[-1:]:
  base_url = 'https://api.telegram.org/bot2049051709:AAGOApTYM_4ZMiSIpt_lvj4QDGrN8_X8MYc/sendMessage?chat_id=-542715306&text= "{}" :: @Banknifty (Short Activated by 200DMA)'.format(sell)
  requests.get(base_url)
  # print(base_url)

In [ ]:
for buy in D_Green[-1:]:
  base_url = 'https://api.telegram.org/bot2049051709:AAGOApTYM_4ZMiSIpt_lvj4QDGrN8_X8MYc/sendMessage?chat_id=-542715306&text= "{}" :: @Banknifty (Long Activated by 200DMA)'.format(buy)
  requests.get(base_url)
  # print(base_url)

In [ ]:
for buy in Green[-1:]:
  base_url = 'https://api.telegram.org/bot2049051709:AAGOApTYM_4ZMiSIpt_lvj4QDGrN8_X8MYc/sendMessage?chat_id=-542715306&text= "{}" :: @Banknifty (Long Activated by 20DMA)'.format(buy)
  requests.get(base_url)
  # print(base_url)

In [ ]:
# Visually Show the close price with the DMA & Supertrend Crossover's Buy And Sell Signals
plt.figure(figsize=(18,9))
plt.title('Close Price History w/ Buy & Sell Signals',fontsize=18)
plt.plot(df['close'],alpha= 0.3, label = 'Close')

plt.scatter(df.index,df['Buy_Signal_Price'],alpha=1,label='Buy Signal',marker='^',color='green')
plt.scatter(df.index,df['Sell_Signal_Price'],alpha=1,label='Sell Signal',marker='v',color='red')

plt.scatter(df.index,df['Buy200'],alpha=1,label='Buy Signal',marker='o',color='blue')
plt.scatter(df.index,df['Sell200'],alpha=1,label='Sell Signal',marker='o',color='black')

plt.ylabel('Close Price',fontsize=18)
plt.show()

In [ ]:
# import plotly.graph_objects as go
# df = df.set_index(pd.DatetimeIndex(df['timestamp'].values))

In [ ]:
# create candlestick
# figure = go.Figure(data=[go.Candlestick(x = df.index,open=df['open'],high=df['high'],low= df['low'],close=df['close'],increasing_line_color='green',decreasing_line_color='red')])

# figure.show()